In [1]:
import os
import re
import math
import copy
import pickle
import shutil
import warnings
import datetime
import numpy as np
import pandas as pd
import tensorflow as tf

In [2]:
import sys
sys.path.append("C:/Users/begas/Project/python/SmartFarm/5. SRC") # 다른 경로의 Class 불러올 때 사용
import common_preprocess_tools as prep
import pred_tools as pred

C:\Users\begas\Anaconda3\lib\site-packages\dask\dataframe\utils.py:14: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [3]:
# -----------------------------------------------|
# ----- Step 1. Settings ------------------------
# -----------------------------------------------|
os.chdir("C:/Users/begas/Project/python/SmartFarm/")
os.getcwd()

home_path  = os.getcwd()
data_path  = os.path.join(home_path,"1. DAT")
save_path  = os.path.join(home_path,"2. OUT")
model_path = os.path.join(home_path,"3. MODEL")

os.makedirs(data_path,exist_ok=True)
os.makedirs(save_path,exist_ok=True)
os.makedirs(model_path,exist_ok=True)

data_files = os.path.join(data_path,os.listdir(data_path)[6])
print(data_files)

working_date = '20220926'

C:\Users\begas\Project\python\SmartFarm\1. DAT\22_이엔티DB_신천농장_외부.csv


### 데이터 불러오기

In [4]:
'''
data_files  : Raw 데이터 위치
time_grp    : 시간대별 통계량 요약 기준 단위(ex : if time_grp = 60, then 60분 단위로 통계량 요약)
timr_window : 시차 (예측시간으로부터 window_v시간 전의 데이터를 설명변수로 사용)
'''
_prep = prep.common_preprocess(data_files = data_files, time_grp = 60, time_window = [12,24])

In [5]:
_prep.load_fn()

>> Raw 데이터 호출 및 단일값 컬럼 제거
모든 값이 단일값인 변수 제거 : 온도/환기팬


,시간,암모니아,황화수소,습도
0,2022-01-01 00:23:00,0.0,0.1,49.0
1,2022-01-01 00:23:00,0.0,0.1,49.0
2,2022-01-01 00:23:00,0.0,0.2,49.0
3,2022-01-01 00:23:00,0.0,0.1,49.0
4,2022-01-01 00:23:00,0.0,0.1,49.0
...,...,...,...,...
42365,2022-01-16 15:32:00,2.4,0.1,43.0
42366,2022-01-16 15:32:00,2.4,0.1,43.0
42367,2022-01-16 15:32:00,2.4,0.1,43.0
42368,2022-01-16 15:32:00,2.4,0.1,43.0


### 예측 데이터 전처리 하기

In [6]:
_prep.preprocess_fn()

>> 이상치, 결측치 처리 및 시간대별 요약 통계량 생성
Step1. 이상치를 허용범위 내로 보정
Step2. 시간 변수를 60 분 단위로 변경
Step3. 시간별 요약통계량 데이터 생성
Step4. 특정 시간대의 데이터가 비어있을 시 해당 시간대 생성
- 22개의 시간대가 비어있습니다. 해당 시간대를 생성합니다.
Step5. 선형 보간법 적용


,시간,암모니아_mean,황화수소_mean,습도_mean,암모니아_min,황화수소_min,습도_min,암모니아_max,황화수소_max,습도_max,암모니아_std,황화수소_std,습도_std
0,2022-01-01 00:00:00,0.083333,0.140000,49.000000,0.0,0.1,49.0,0.4,0.2,49.0,0.163128,0.049195,0.000000
1,2022-01-01 01:00:00,0.013333,0.137500,49.016667,0.0,0.1,49.0,0.4,0.2,50.0,0.072103,0.048615,0.128556
2,2022-01-01 02:00:00,0.000000,0.135000,49.183333,0.0,0.1,49.0,0.0,0.2,50.0,0.000000,0.047897,0.388562
3,2022-01-01 03:00:00,0.000000,0.140000,50.000000,0.0,0.1,50.0,0.0,0.2,50.0,0.000000,0.049195,0.000000
4,2022-01-01 04:00:00,0.000000,0.135833,49.866667,0.0,0.1,49.0,0.0,0.2,50.0,0.000000,0.048152,0.341360
...,...,...,...,...,...,...,...,...,...,...,...,...,...
371,2022-01-16 11:00:00,1.223333,0.113333,50.375000,1.2,0.1,50.0,1.6,0.2,51.0,0.094142,0.034136,0.486153
372,2022-01-16 12:00:00,1.286667,0.116667,50.000000,1.2,0.1,50.0,1.6,0.2,50.0,0.165480,0.037424,0.000000
373,2022-01-16 13:00:00,1.456667,0.110000,47.375000,1.2,0.1,47.0,1.6,0.2,50.0,0.192609,0.030126,0.996317
374,2022-01-16 14:00:00,1.913333,0.106667,46.416667,1.6,0.1,46.0,2.0,0.2,47.0,0.165480,0.025049,0.495074


### 예측 - 분석용 데이터 생성

In [7]:
_prep.pred_anal_dat_fn()

>> 학습 - 분석용 데이터 생성[종속, 설명(시차변수) 생성]
Step1. 목표변수: 암모니아, 황화수소 단위별 max 값
Step2. 시차 변수 생성
Step3. base_time 컬럼 생성


{'pred':               base_time        predict_time  x_암모니아_mean_12  x_황화수소_mean_12  \
 0   2022-01-02 00:00:00 2022-01-02 12:00:00        1.200000        0.122500   
 1   2022-01-02 01:00:00 2022-01-02 13:00:00        1.216667        0.116667   
 2   2022-01-02 02:00:00 2022-01-02 14:00:00        1.233333        0.115833   
 3   2022-01-02 03:00:00 2022-01-02 15:00:00        1.256667        0.118333   
 4   2022-01-02 04:00:00 2022-01-02 16:00:00        1.300000        0.116667   
 ..                  ...                 ...             ...             ...   
 347 2022-01-16 11:00:00 2022-01-16 23:00:00        1.223333        0.113333   
 348 2022-01-16 12:00:00 2022-01-17 00:00:00        1.286667        0.116667   
 349 2022-01-16 13:00:00 2022-01-17 01:00:00        1.456667        0.110000   
 350 2022-01-16 14:00:00 2022-01-17 02:00:00        1.913333        0.106667   
 351 2022-01-16 15:00:00 2022-01-17 03:00:00        2.446667        0.102500   
 
      x_습도_mean_12  x_암모니아_min

### 예측 하기

In [8]:
final_result = []
for y_name in ['y_암모니아','y_황화수소']:
    print(f'Model Train Date : {working_date} / Y Name : {y_name}')
    _pred = pred.pred(working_date, y_name, model_path, _prep.pred_anal_dat)
    predict_result = _pred.pred_ret.copy()
    predict_result['y_name'] = y_name
    final_result.append(predict_result)
final_result = pd.concat(final_result, ignore_index=True)
final_result

Model Train Date : 20220926 / Y Name : y_암모니아
Working Date : 20220926 / Y Name : y_암모니아 / BEST MODEL is ARIMA
Model Train Date : 20220926 / Y Name : y_황화수소
Working Date : 20220926 / Y Name : y_황화수소 / BEST MODEL is LGB


,base_time,predict_time,x_암모니아_mean_12,x_황화수소_mean_12,x_습도_mean_12,x_암모니아_min_12,x_황화수소_min_12,x_습도_min_12,x_습도_max_12,x_암모니아_std_12,...,x_습도_mean_24,x_암모니아_min_24,x_황화수소_min_24,x_습도_min_24,x_습도_max_24,x_암모니아_std_24,x_황화수소_std_24,x_습도_std_24,pred_val,y_name
0,2022-01-02 00:00:00,2022-01-02 12:00:00,1.200000,0.122500,48.750000,1.2,0.1,46.0,49.0,4.459512e-16,...,49.425000,0.8,0.1,49.0,50.0,1.926085e-01,0.041370,0.496416,0.324319,y_암모니아
1,2022-01-02 01:00:00,2022-01-02 13:00:00,1.216667,0.116667,46.000000,1.2,0.1,46.0,46.0,8.026567e-02,...,45.691667,1.2,0.1,45.0,49.0,4.459512e-16,0.038856,0.719428,0.780233,y_암모니아
2,2022-01-02 02:00:00,2022-01-02 14:00:00,1.233333,0.115833,46.000000,1.2,0.1,46.0,46.0,1.110177e-01,...,45.504202,1.2,0.1,45.0,46.0,1.814970e-01,0.027754,0.502096,0.991558,y_암모니아
3,2022-01-02 03:00:00,2022-01-02 15:00:00,1.256667,0.118333,45.633333,1.2,0.1,2.0,46.0,1.400680e-01,...,42.050000,2.0,0.1,41.0,43.0,1.755663e-01,0.026450,0.786974,1.789949,y_암모니아
4,2022-01-02 04:00:00,2022-01-02 16:00:00,1.300000,0.116667,46.000000,1.2,0.1,46.0,46.0,1.739313e-01,...,42.250000,2.4,0.1,42.0,43.0,1.719553e-01,0.026450,0.434828,1.840144,y_암모니아
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
699,2022-01-16 11:00:00,2022-01-16 23:00:00,1.223333,0.113333,50.375000,1.2,0.1,50.0,51.0,9.414215e-02,...,46.000000,1.6,0.1,46.0,46.0,1.758214e-01,0.030126,0.000000,0.195126,y_황화수소
700,2022-01-16 12:00:00,2022-01-17 00:00:00,1.286667,0.116667,50.000000,1.2,0.1,50.0,50.0,1.654804e-01,...,46.008333,1.6,0.1,46.0,47.0,1.698904e-01,0.031210,0.091287,0.198826,y_황화수소
701,2022-01-16 13:00:00,2022-01-17 01:00:00,1.456667,0.110000,47.375000,1.2,0.1,47.0,50.0,1.926085e-01,...,46.358333,1.6,0.1,46.0,47.0,8.026567e-02,0.027754,0.481521,0.199610,y_황화수소
702,2022-01-16 14:00:00,2022-01-17 02:00:00,1.913333,0.106667,46.416667,1.6,0.1,46.0,47.0,1.654804e-01,...,46.775000,1.2,0.1,46.0,47.0,1.225317e-01,0.034136,0.419333,0.191638,y_황화수소
